# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 1 2022 10:07AM,245509,19,ACG-2102491090,ACG North America LLC,Released
1,Sep 1 2022 10:02AM,245508,10,CTF0011389,"Citieffe, Inc.",Released
2,Sep 1 2022 10:02AM,245508,10,8471147,"Citieffe, Inc.",Released
3,Sep 1 2022 9:44AM,245507,20,8470067,"Exact-Rx, Inc.",Released
4,Sep 1 2022 9:44AM,245507,20,8470068,"Exact-Rx, Inc.",Released
5,Sep 1 2022 9:44AM,245507,20,8470069,"Exact-Rx, Inc.",Released
6,Sep 1 2022 9:44AM,245507,20,8470070,"Exact-Rx, Inc.",Released
7,Sep 1 2022 9:41AM,245506,10,PRF-36217,Bio-PRF,Released
8,Sep 1 2022 9:41AM,245506,10,PRF-36233,Bio-PRF,Released
9,Sep 1 2022 9:41AM,245506,10,PRF-36243,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,245504,Released,2
20,245506,Released,9
21,245507,Released,4
22,245508,Released,2
23,245509,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245504,NaN,NaN,2.0
245506,NaN,NaN,9.0
245507,NaN,NaN,4.0
245508,NaN,NaN,2.0
245509,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245419,1.0,0.0,29.0
245470,0.0,0.0,1.0
245474,0.0,2.0,5.0
245479,0.0,0.0,1.0
245480,0.0,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245419,1,0,29
245470,0,0,1
245474,0,2,5
245479,0,0,1
245480,0,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245419,1,0,29
1,245470,0,0,1
2,245474,0,2,5
3,245479,0,0,1
4,245480,0,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245419,1,,29
1,245470,,,1
2,245474,,2,5
3,245479,,,1
4,245480,,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 1 2022 10:07AM,245509,19,ACG North America LLC
1,Sep 1 2022 10:02AM,245508,10,"Citieffe, Inc."
3,Sep 1 2022 9:44AM,245507,20,"Exact-Rx, Inc."
7,Sep 1 2022 9:41AM,245506,10,Bio-PRF
16,Sep 1 2022 9:30AM,245504,10,"Senseonics, Incorporated"
18,Sep 1 2022 9:27AM,245503,12,Hush Hush
19,Sep 1 2022 9:21AM,245502,12,Hush Hush
46,Sep 1 2022 9:12AM,245500,10,"Herbivore Botanicals, LLC."
47,Sep 1 2022 9:05AM,245499,10,"Methapharm, Inc."
55,Sep 1 2022 9:04AM,245497,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 1 2022 10:07AM,245509,19,ACG North America LLC,,,1
1,Sep 1 2022 10:02AM,245508,10,"Citieffe, Inc.",,,2
2,Sep 1 2022 9:44AM,245507,20,"Exact-Rx, Inc.",,,4
3,Sep 1 2022 9:41AM,245506,10,Bio-PRF,,,9
4,Sep 1 2022 9:30AM,245504,10,"Senseonics, Incorporated",,,2
5,Sep 1 2022 9:27AM,245503,12,Hush Hush,,,1
6,Sep 1 2022 9:21AM,245502,12,Hush Hush,,24,3
7,Sep 1 2022 9:12AM,245500,10,"Herbivore Botanicals, LLC.",,,1
8,Sep 1 2022 9:05AM,245499,10,"Methapharm, Inc.",,7,1
9,Sep 1 2022 9:04AM,245497,10,ISDIN Corporation,,,17


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 10:07AM,245509,19,ACG North America LLC,1,,
1,Sep 1 2022 10:02AM,245508,10,"Citieffe, Inc.",2,,
2,Sep 1 2022 9:44AM,245507,20,"Exact-Rx, Inc.",4,,
3,Sep 1 2022 9:41AM,245506,10,Bio-PRF,9,,
4,Sep 1 2022 9:30AM,245504,10,"Senseonics, Incorporated",2,,
5,Sep 1 2022 9:27AM,245503,12,Hush Hush,1,,
6,Sep 1 2022 9:21AM,245502,12,Hush Hush,3,24,
7,Sep 1 2022 9:12AM,245500,10,"Herbivore Botanicals, LLC.",1,,
8,Sep 1 2022 9:05AM,245499,10,"Methapharm, Inc.",1,7,
9,Sep 1 2022 9:04AM,245497,10,ISDIN Corporation,17,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 10:07AM,245509,19,ACG North America LLC,1,,
1,Sep 1 2022 10:02AM,245508,10,"Citieffe, Inc.",2,,
2,Sep 1 2022 9:44AM,245507,20,"Exact-Rx, Inc.",4,,
3,Sep 1 2022 9:41AM,245506,10,Bio-PRF,9,,
4,Sep 1 2022 9:30AM,245504,10,"Senseonics, Incorporated",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 10:07AM,245509,19,ACG North America LLC,1.0,NaN,NaN
1,Sep 1 2022 10:02AM,245508,10,"Citieffe, Inc.",2.0,NaN,NaN
2,Sep 1 2022 9:44AM,245507,20,"Exact-Rx, Inc.",4.0,NaN,NaN
3,Sep 1 2022 9:41AM,245506,10,Bio-PRF,9.0,NaN,NaN
4,Sep 1 2022 9:30AM,245504,10,"Senseonics, Incorporated",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 1 2022 10:07AM,245509,19,ACG North America LLC,1.0,0.0,0.0
1,Sep 1 2022 10:02AM,245508,10,"Citieffe, Inc.",2.0,0.0,0.0
2,Sep 1 2022 9:44AM,245507,20,"Exact-Rx, Inc.",4.0,0.0,0.0
3,Sep 1 2022 9:41AM,245506,10,Bio-PRF,9.0,0.0,0.0
4,Sep 1 2022 9:30AM,245504,10,"Senseonics, Incorporated",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2700441,130.0,9.0,0.0
12,491005,4.0,24.0,0.0
15,981861,59.0,0.0,1.0
19,491002,1.0,1.0,0.0
20,245507,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2700441,130.0,9.0,0.0
1,12,491005,4.0,24.0,0.0
2,15,981861,59.0,0.0,1.0
3,19,491002,1.0,1.0,0.0
4,20,245507,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,130.0,9.0,0.0
1,12,4.0,24.0,0.0
2,15,59.0,0.0,1.0
3,19,1.0,1.0,0.0
4,20,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,130.0
1,12,Released,4.0
2,15,Released,59.0
3,19,Released,1.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20
Status,,,,,
Completed,0.0,0.0,1.0,0.0,0.0
Executing,9.0,24.0,0.0,1.0,0.0
Released,130.0,4.0,59.0,1.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20
0,Completed,0.0,0.0,1.0,0.0,0.0
1,Executing,9.0,24.0,0.0,1.0,0.0
2,Released,130.0,4.0,59.0,1.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20
0,Completed,0.0,0.0,1.0,0.0,0.0
1,Executing,9.0,24.0,0.0,1.0,0.0
2,Released,130.0,4.0,59.0,1.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()